In [1]:
import csv
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.stats import entropy
import numpy as np
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test_id.csv to test_id.csv
Saving test_ood.csv to test_ood.csv
Saving train.csv to train.csv


In [4]:
train_df = pd.read_csv('train.csv')
test_id = pd.read_csv('test_id.csv')
test_ood=pd.read_csv('test_ood.csv')

test_id=test_id.drop(test_id.columns[[0]],axis=1)
test_ood=test_ood.drop(test_ood.columns[[0,2,3]],axis=1)

X_train = train_df.drop(train_df.columns[[0,1]],axis=1)
y_train = train_df['Label']

X_test_id = test_id.drop(test_id.columns[[0]],axis=1)
X_test_ood = test_ood.drop(test_ood.columns[[0]],axis=1)
y_test_id = test_id['Label']
y_test_ood = test_ood['Label']

In [5]:
len(test_ood)

938

In [6]:
n_estimators = 100
model = RandomForestClassifier(n_estimators=n_estimators, random_state = 42)
model.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [7]:
pred_id = model.predict(X_test_id)
pred_ood = model.predict(X_test_ood)

prob_id = model.predict_proba(X_test_id)[:,1]
prob_ood = model.predict_proba(X_test_ood)[:,1]

In [8]:
def highentropy(threshold,proba):
  return True if abs(proba-0.5)<threshold else False

In [34]:
threshold = 0.25 # entropy within median (0.5); lower threshold obtains higher entropy points

he_id_truval = [y_test_id[j] for j,n in enumerate(prob_id) if highentropy(threshold,n)] # use to train heid_selective_model
he_id_fts = [X_test_id.iloc[j] for j,n in enumerate(prob_id) if highentropy(threshold,n)]
le_id_truval = [y_test_id[j] for j,n in enumerate(prob_id) if highentropy(threshold,n)==False] # use to train leid_selective_model
le_id_fts = [X_test_id.iloc[j] for j,n in enumerate(prob_id) if highentropy(threshold,n)==False]
he_id_predval = [pred_id[j] for j,n in enumerate(prob_id) if highentropy(threshold,n)]
le_id_predval = [pred_id[j] for j,n in enumerate(prob_id) if highentropy(threshold,n)==False]

he_ood_truval = [y_test_ood[j] for j,n in enumerate(prob_ood) if highentropy(threshold,n)]
he_ood_fts = [X_test_ood.iloc[j] for j,n in enumerate(prob_ood) if highentropy(threshold,n)]
le_ood_truval = [y_test_ood[j] for j,n in enumerate(prob_ood) if highentropy(threshold,n)==False]
le_ood_fts = [X_test_ood.iloc[j] for j,n in enumerate(prob_ood) if highentropy(threshold,n)==False]
#he_ood_predval = [pred_ood[j] for j,n in enumerate(prob_ood) if highentropy(threshold,n)]
#le_ood_predval = [pred_ood[j] for j,n in enumerate(prob_ood) if highentropy(threshold,n)==False]


heid_selective_model = LogisticRegression(random_state=42, max_iter=5000)
heid_selective_model.fit(he_id_fts,he_id_truval)

leid_selective_model = LogisticRegression(random_state=42, max_iter=5000)
leid_selective_model.fit(le_id_fts,le_id_truval)


#heood_selective_model = LogisticRegression(random_state=42)
#heood_selective_model.fit(he_ood_fts,he_ood_truval)

#leood_selective_model = LogisticRegression(random_state=42)
#leood_selective_model.fit(le_ood_fts,le_ood_truval)


he_ood_predictions = heid_selective_model.predict(X_test_ood)
le_ood_predictions = leid_selective_model.predict(X_test_ood)

he_acc = accuracy_score(he_ood_predictions, y_test_ood)
le_acc = accuracy_score(le_ood_predictions, y_test_ood)

print(he_acc)
print(le_acc)

0.5618336886993603
0.6119402985074627


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [31]:
len(le_id_fts)

85

In [35]:
np.mean([he_acc,le_acc])

0.5868869936034116

In [33]:
#benchmark
aggregate = LogisticRegression(random_state=42, max_iter=5000)
aggregate.fit(X_test_id, y_test_id)

preds = aggregate.predict(X_test_ood)
accuracy_score(preds, y_test_ood)

0.5810234541577826